# Testing rechunking vs repacking training

Starting from scratch

In [1]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v6-validation"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v6/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/trainer-v6-validation
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v6
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


In [15]:
# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer 6 --n_embd 1024 \
        --vocab_size world --skip-if-exists \
        "../model/L6-D1024-world-v6base-init.pth"

[2024-04-10 21:54:59,080] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
---- Initializing model ----
No of layers: 6
Embedding size: 1024
Output model path: ../model/L6-D1024-world-v6base-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Output model exists, skipping init_model


# Dataset preloading

In [10]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml"

Map (num_proc=160): 100%|███| 1000000/1000000 [00:48<00:00, 20744.44 examples/s]
Filter (num_proc=160): 100%|█| 1000000/1000000 [00:06<00:00, 164014.77 examples/
Map (num_proc=160): 100%|█████████| 10020/10020 [00:12<00:00, 800.34 examples/s]
Saving the dataset (4/4 shards): 100%|█| 10020/10020 [00:12<00:00, 771.38 exampl
Saving the dataset (1/1 shards): 100%|█████| 2/2 [00:00<00:00, 78.89 examples/s]


In [11]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-packing.yaml"

Map (num_proc=160): 100%|███| 1000000/1000000 [00:44<00:00, 22469.77 examples/s]
Filter (num_proc=160): 100%|█| 1000000/1000000 [00:06<00:00, 166543.29 examples/
Map (num_proc=160): 100%|██████| 134691/134691 [00:19<00:00, 7044.24 examples/s]
Saving the dataset (4/4 shards): 100%|█| 10026/10026 [00:13<00:00, 716.74 exampl
Saving the dataset (1/1 shards): 100%|█████| 2/2 [00:00<00:00, 80.41 examples/s]


# With rechunking

In [6]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L6-D1024-world-v6base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-32k-rechunk/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Rechunk 32k - (deepspeed_stage_1)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=4 \
        --trainer.devices="{GPU_DEVICES}"

[2024-04-11 04:53:40,788] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v6-validation/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L6-D1024-world-v6base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-32k-rechunk/', '--trainer.logger.init_args.name=infctx-v6-validation - Rechunk 32k - (deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.microbatch_size=4', '--trainer.devices=auto'], args=['fit', '-c', '/home/recursal/RW

# With dataset packing

In [9]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-packing.yaml" \
        --model.load_model="../model/L6-D1024-world-v6base-init.pth" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-32k-packing/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Packing 32k - (deepspeed_stage_1)" \
        --trainer.strategy="deepspeed_stage_1" \
        --trainer.microbatch_size=4 \
        --trainer.devices="{GPU_DEVICES}"

[2024-04-11 05:49:37,197] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v6-validation/config/enwiki_100k-world-32k-packing.yaml', '--model.load_model=../model/L6-D1024-world-v6base-init.pth', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-32k-packing/', '--trainer.logger.init_args.name=infctx-v6-validation - Packing 32k - (deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.microbatch_size=4', '--trainer.devices=auto'], args=['fit', '-c', '/home/recursal/RW